<a href="https://colab.research.google.com/github/justinfmccarty/bipv_database/blob/main/extract_bipv_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Eiffert and Kiss - Building Integrated Photovoltaic Design Book

Eiffert, Patrina, and Gregory J Kiss. “Building-Integrated Photovoltaic Designs for Commercial and Institutional Structures A Sourcebook for Architects,” 2000, 92.


In [ ]:
print('hello')

hello
